Setup - run first

In [ ]:
import iris
import pandas as pd

host = "localhost"
# this is the superserver port
port = 32782
namespace = "FHIRSERVER"
user = "_SYSTEM"
password = "SYS"

conn = iris.connect(
    hostname=host,
    port=port,
    namespace=namespace,
    username=user,
    password=password
)

# create a cursor
cursor = conn.cursor()
cursor

The diabetes example works with the Patient table in IRIS. This doesn't expose extensions which we need to get race and ethnic data.
So instead we use the raw SQL table `HSFHIR_X0001_R.Rsrc where ResourceType = 'Patient'`, restricting this to Patient relatad data.

The returned functions uses two functions which have been added to IRIS.
- **GetJSON** which returns data at a specific element
- **GetFHIRPath** which returns data based on a FHIR Path expression.

Running the example shows the data returned from both is the same.

In [3]:


sql = """
      select top 10
ID
           , Key
 , GetJSON(ResourceString,'extension') Extension
 , GetFHIRPath(ResourceString, 'Patient.extension') ExtensionFHIRPath
      from HSFHIR_X0001_R.Rsrc where ResourceType = 'Patient'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ID,Key,Extension,ExtensionFHIRPath
0,2,Patient/2,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
1,3,Patient/3,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
2,4,Patient/4,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
3,5,Patient/5,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
4,6,Patient/6,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
5,38,Patient/38,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
6,2369,Patient/2369,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
7,2370,Patient/2370,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
8,2371,Patient/2371,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
9,2372,Patient/2372,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."


For data analysis and machine learning purposes we want to work with values not JSON.

We have a couple of options, we could use python libraries [FHIR Resources](https://pypi.org/project/fhir.resources/) to help with extracting the values, or we use [FHIRPath](https://build.fhir.org/ig/HL7/FHIRPath/).

We are not going to go into the detail of FHIRPath as this can be a lengthy topic. I would recommend using online FHIRPath tools such [fhirpath.js](https://hl7.github.io/fhirpath.js/) to explore how this works.

I used this together with a sample FHIR Patient, to create the following:

- Race
  - Patient.extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race')
- Ethnicity
  - Patient.extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity')
- Birth Sex
  - Patient.extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex')

Using these examples, we can now alter our original query to:

In [4]:
sql = """
      select top 10
ID
           , Key
 , GetFHIRPath(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'')') RaceExtension
 , GetFHIRPath(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'')') EthnicityExtension
 , GetFHIRPath(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'')') BirthSexExtension
      from HSFHIR_X0001_R.Rsrc where ResourceType = 'Patient'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ID,Key,RaceExtension,EthnicityExtension,BirthSexExtension
0,2,Patient/2,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
1,3,Patient/3,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
2,4,Patient/4,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
3,5,Patient/5,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
4,6,Patient/6,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
5,38,Patient/38,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
6,2369,Patient/2369,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
7,2370,Patient/2370,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
8,2371,Patient/2371,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."
9,2372,Patient/2372,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure..."


We still have json results but we have split the extensions up. The BirthSexExtension is probably easier to tackle first as it is a simple extension.

```json
[{"url":"http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex","valueCode":"M"}]
```

We also need to change the function from `GetFHIRPath` to `GetFHIRPathOne`, adding a parameter for the FHIR Resource type.


In [7]:
sql = """
      select top 10
ID
           , Key
 , GetFHIRPath(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'')') RaceExtension
 , GetFHIRPath(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'')') EthnicityExtension
, GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'').valueCode', 'Patient') BirthSex
      from HSFHIR_X0001_R.Rsrc where ResourceType = 'Patient'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ID,Key,RaceExtension,EthnicityExtension,BirthSex
0,2,Patient/2,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M
1,3,Patient/3,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",F
2,4,Patient/4,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M
3,5,Patient/5,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",F
4,6,Patient/6,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",F
5,38,Patient/38,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",F
6,2369,Patient/2369,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M
7,2370,Patient/2370,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M
8,2371,Patient/2371,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M
9,2372,Patient/2372,"[{""url"":""http://hl7.org/fhir/us/core/Structure...","[{""url"":""http://hl7.org/fhir/us/core/Structure...",M


Both race and ethnicity are complex extensions, they are extensions with extensions. So once we have found the extension, we need to find the sub extension. We do this by including extension again in the FHIRPath.

- Race
  - Patient.extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').valueCoding.code
- Ethnic
  - Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'').extension(''ombCategory'').valueCoding.code

Note the inclusion of both code and display in the results.




In [8]:
sql = """
      select top 10
ID
           , Key
 , GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'').extension(''ombCategory'').valueCoding.display','Patient') Race
 , GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'').extension(''ombCategory'').valueCoding.code','Patient') RaceCode
 , GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'').extension(''ombCategory'').valueCoding.display','Patient') Ethnic
 , GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'').extension(''ombCategory'').valueCoding.code','Patient') EthnicCode
 , GetFHIRPathOne(ResourceString, 'Patient.extension(''http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'').valueCode', 'Patient') BirthSex
      from HSFHIR_X0001_R.Rsrc where ResourceType = 'Patient'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ID,Key,Race,RaceCode,Ethnic,EthnicCode,BirthSex
0,2,Patient/2,White,2106-3,Not Hispanic or Latino,2186-5,M
1,3,Patient/3,White,2106-3,Not Hispanic or Latino,2186-5,F
2,4,Patient/4,White,2106-3,Not Hispanic or Latino,2186-5,M
3,5,Patient/5,Asian,2028-9,Not Hispanic or Latino,2186-5,F
4,6,Patient/6,Asian,2028-9,Not Hispanic or Latino,2186-5,F
5,38,Patient/38,White,2106-3,Not Hispanic or Latino,2186-5,F
6,2369,Patient/2369,Asian,2028-9,Not Hispanic or Latino,2186-5,M
7,2370,Patient/2370,White,2106-3,Not Hispanic or Latino,2186-5,M
8,2371,Patient/2371,White,2106-3,Not Hispanic or Latino,2186-5,M
9,2372,Patient/2372,White,2106-3,Hispanic or Latino,2135-2,M
